# Modification du code de pyalgotrade
## Ajout des contrats 

La méthodologie qui va être utilisée est d'ajouter les contrat et les méthodes utilisant les contrats dans backtesting.py

exemple contenu dans backtesting.py (Pyalgotrade)

In [ ]:
def createMarketOrder(self, action, instrument, quantity, onClose=False):
        # In order to properly support market-on-close with intraday feeds I'd need to know about different
        # exchange/market trading hours and support specifying routing an order to a specific exchange/market.
        # Even if I had all this in place it would be a problem while paper-trading with a live feed since
        # I can't tell if the next bar will be the last bar of the market session or not.
        if onClose is True and self.__barFeed.isIntraday():
            raise Exception("Market-on-close not supported with intraday feeds")

        return MarketOrder(action, instrument, quantity, onClose, self.getInstrumentTraits(instrument))

Exemple comtenu dans IbBroker.py (le code de débogage à été retiré)

In [ ]:
def createMarketOrder(self, contract,action, quantity, GoodTillCanceled = True,AllOrNone = True):
        ibContract=contract
        
        ibOrder=Order()
        ibOrder.m_orderId       = self.__getUniqueOrderId()
        ibOrder.m_totalQuantity = quantity
        
        if action == 'BUY':
            ibOrder.m_action    = 'BUY'
        elif action == 'SELL':
            ibOrder.m_action = 'SELL'
 
        ibOrder.m_orderType = 'MKT'
 
        if AllOrNone == AllOrNone:
            ibOrder.m_allOrNone = 1
        else:
            ibOrder.m_allOrNone = 0

        if GoodTillCanceled == True:
            ibOrder.m_tif = 'GTC'
        else:
            ibOrder.m_tif = 'DAY'
            
        self.__setOrderStatus(contract=contract,order=order,status='GENERATED', datetime=datetime.datetime.now())
        self.__registerOrder(contract=contract,order=order,status='GENERATED',datetime=datetime.datetime.now())
        if self.__debug:
            print('%s[IB LiveBroker createMarketOrder]INCREASE ORDER ID: %s' %(now,self.__nextOrderId))

Pour tester l'utilisation des contrats, la stratégie "tutorial-4.py" sera utilisée, puisqu'elle fait plusieurs transaction d'action. En premier lieux les contrats de stock sera implémentés, puis lorsque fonctionnel la gestion des autres typs de transaction pouront être ajoutées

Première modification, ajout du order "OptionOrder" basé sur marketorder, dans backtesting.py

In [ ]:
class MarketOrder(broker.MarketOrder, BacktestingOrder):
    def __init__(self, action, instrument, quantity, onClose, instrumentTraits):
        super(MarketOrder, self).__init__(action, instrument, quantity, onClose, instrumentTraits)

    def process(self, broker_, bar_):
        return broker_.getFillStrategy().fillMarketOrder(broker_, self, bar_)

class OptionOrder(broker.OptionOrder, BacktestingOrder):
    def __init__(self, action, instrument, quantity, onClose, instrumentTraits, right, strike, expiry):
        super(OptionOrder, self).__init__(action, instrument, quantity, onClose, instrumentTraits , right, strike, expiry)

    def process(self, broker_, bar_):
        return broker_.getFillStrategy().fillOptionOrder(broker_, self, bar_)


Deuxième modification, ajout de la méthode createOptionOrder, basée sur createMarketOrder, dans backtesting.py. Cette modification sera aussi appliquée pour limitOrder, stopOrder, stopLImitOrder

In [ ]:
def createMarketOrder(self, action, instrument, quantity, onClose=False):
        # In order to properly support market-on-close with intraday feeds I'd need to know about different
        # exchange/market trading hours and support specifying routing an order to a specific exchange/market.
        # Even if I had all this in place it would be a problem while paper-trading with a live feed since
        # I can't tell if the next bar will be the last bar of the market session or not.
        if onClose is True and self.__barFeed.isIntraday():
            raise Exception("Market-on-close not supported with intraday feeds")

        return MarketOrder(action, instrument, quantity, onClose, self.getInstrumentTraits(instrument))
    
    
def createOptionOrder(self, action, instrument, quantity, onClose=False):
    # In order to properly support market-on-close with intraday feeds I'd need to know about different
    # exchange/market trading hours and support specifying routing an order to a specific exchange/market.
    # Even if I had all this in place it would be a problem while paper-trading with a live feed since
    # I can't tell if the next bar will be the last bar of the market session or not.
    if onClose is True and self.__barFeed.isIntraday():
        raise Exception("Market-on-close not supported with intraday feeds")

    return OptionOrder(action, instrument, quantity, onClose, self.getInstrumentTraits(instrument))


Troisième ajout, Dans "fillStrategy.py", abstract fillOptionOrder, basé sur fillMarketOrder. Elle sera implémentée dans les stratégies plus bas

In [ ]:
@abc.abstractmethod
    def fillMarketOrder(self, broker_, order, bar):
        """Override to return the fill price and quantity for a market order or None if the order can't be filled
        at the given time.

        :param broker_: The broker.
        :type broker_: :class:`Broker`
        :param order: The order.
        :type order: :class:`pyalgotrade.broker.MarketOrder`
        :param bar: The current bar.
        :type bar: :class:`pyalgotrade.bar.Bar`
        :rtype: A :class:`FillInfo` or None if the order should not be filled.
        """
        raise NotImplementedError()
        
    @abc.abstractmethod
    def fillOptionOrder(self, broker_, order, bar):
        """Override to return the fill price and quantity for a market order or None if the order can't be filled
        at the given time.

        :param broker_: The broker.
        :type broker_: :class:`Broker`
        :param order: The order.
        :type order: :class:`pyalgotrade.broker.MarketOrder`
        :param bar: The current bar.
        :type bar: :class:`pyalgotrade.bar.Bar`
        :rtype: A :class:`FillInfo` or None if the order should not be filled.
        """
        raise NotImplementedError()

Quatrième ajout Dans "fillStrategy.py", classe defaultStrategy -> fillOptionOrder, basé sur fillMarketOrder. Elle fait la gestion de l'expiration

Pour faire le controle de l'expiration de l'option, il faut ajouter la date d'expiration à "position" et ses getter et setters

In [ ]:
    __init__
    
        self.__expiryDate = None
    
    def setExpiryDate(self, dateTime):
        self.__expiryDate = dateTime
    
    def getExpiryDate(self):
        return self.__expiryDate